In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Loading the data

In [ ]:
wine_df = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
wine_df.head()

In [ ]:
wine_df.isnull().sum()

In [ ]:
wine_df['quality'].value_counts()

In [ ]:
sns.heatmap(wine_df.corr())
plt.title('Correlations between features')
plt.show()

In [ ]:
wine_df.corr()['quality']

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
wine_df_std = scaler.fit_transform(wine_df)
wine_df_std = pd.DataFrame(wine_df_std, columns=wine_df.columns)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wine_df_std.drop(columns='quality'), wine_df_std['quality'], test_size= 0.2, random_state=42)
reg = KNeighborsRegressor()

In [ ]:
num_features = []
train_score = []
test_score = []

for k in range(2, 12):
    selector = SelectKBest(f_regression, k = k)
    X_train_new = selector.fit_transform(X_train, y_train)
    X_test_new = selector.transform(X_test)
    reg.fit(X_train_new, y_train)
    num_features.append(k)
    train_score.append(reg.score(X_train_new, y_train))
    test_score.append(reg.score(X_test_new, y_test))

sns.lineplot(x = num_features, y = train_score)
sns.lineplot(x = num_features, y = test_score)
plt.show()

In [ ]:
selector = SelectKBest(f_regression, k = 5)
X_train_new = selector.fit_transform(X_train, y_train)
X_test_new = selector.transform(X_test)
reg.fit(X_train_new, y_train)
print(reg.score(X_test_new, y_test))

In [ ]:
params = {'n_neighbors': range(2, 45), 'weights':['uniform','distance']}
gs = GridSearchCV(estimator=reg, param_grid=params, cv = 3)
gs.fit(X_train_new, y_train)
reg2 = gs.best_estimator_

In [ ]:
from sklearn.metrics import r2_score
reg2.fit(X_train_new, y_train)
y_pred = reg2.predict(X_test_new)
r2 = r2_score(y_test, y_pred)
print(r2 ** (1/2))

**This is my first time submitting a task on Kaggle. Please leave a comment to improve my skills, and also thank you!**